In [58]:
import csv
import pandas as pd
import mysql.connector
from datetime import datetime
import json
import requests

In [59]:
# connecting to the mysql database on localhost using mysql.connector
mydb = mysql.connector.connect(
    host="localhost",
    database="moviehub",
    user="root",
    password="mysqlpassword"
)
mycursor = mydb.cursor()

In [60]:
# Queries to insert data from different csv files into the database
# Queries to insert imdb data into the database
imovie_Query = "INSERT INTO IMDBMovie(primary_title, original_title, release_date, runtime, is_adult, imdb_link, genres) VALUES (%s, %s, %s, %s, %s, %s, %s)"
iperson_Query = "INSERT INTO IMDBPerson(mid, name, birthday, deathday, primary_profession, known_for, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s)"
irating_Query = "INSERT INTO IMDBRatings(id,rating,votes) VALUES (%s, %s, %s)"

# Queries to insert tmdb data into the database
tmovie_Query = "INSERT INTO TMDBMovie(title, original_title, release_date, runtime, is_adult, imdb_link, overview, popularity, poster_path, collection_belongs_to, revenue, budget, tagline, homepage_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
tperson_Query = "INSERT INTO TMDBPerson(id, name, birthday, deathday, alias_name, gender, biography, profile_path, popularity, place_of_birth, homepage, home_country, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Queries to insert timdb data into the database
timovie_Query = "INSERT INTO TIMDBMovie(movieid, title, release_date, imdb_link) VALUES (%s, %s, %s, %s)"
tirating_Query = "INSERT INTO TIMDBRatings(movieid,imdb_link,rating) VALUES (%s, %s, %s)"

# Queries to insert rotten tomatoes data into the database
rtmovie_query = "INSERT INTO RTMovie(title, cast1, cast2, description, director, genre, rating) VALUES (%s, %s, %s, %s, %s, %s, %s)"

# Queries to insert movielens data into the database
mlmovie_Query = "INSERT INTO MLDBMovie(movieid, title, release_date, imdb_link) VALUES (%s, %s, %s, %s)"
mlrating_Query = "INSERT INTO MLDBRatings(movieid,imdb_link,rating) VALUES (%s, %s, %s)"

# Queries to insert celebrity api person data into the database
capiperson_query = "INSERT INTO CAPIPerson(name,birthday,occupation,gender,nationality) VALUES (%s, %s, %s, %s, %s)"

# Query to insert mapping data using movielens, imdb and tmdb data into the database
dbmapping_Query = "INSERT INTO Mapping(movieId,imdbId,tmdbId,timdbId) VALUES (%s, %s, %s,%s)"

In [61]:
# open links.csv and store each row in a dictionary with movieid as key and imdbid as values
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/timdb/links.csv', 'r', encoding='utf8') as f:
    fopen = csv.reader(f, delimiter=',')
    imdbid_dict = {}
    for rdata in fopen:
        if rdata[0] == 'movie_id':
            continue
        imdbid_dict[rdata[0]] = rdata[1]

In [22]:
# inserting timdb movie data in timdbmovie table
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/timdb/titles.csv', 'r', encoding='utf8') as f:
    fopen = csv.reader(f, delimiter=',')
    for rdata in fopen:
        if rdata[0] == 'movie_id':
            continue
        # Extracting data from the csv file
        movieid = rdata[0]
        title = rdata[1][:-7]
        re_d = rdata[1][-6:]
        release_date = re_d[1:-1]
        imdb_link = imdbid_dict[movieid]
        try:
            dtuple = (movieid, title, release_date, imdb_link)
            mycursor.execute(timovie_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ", exc)
print("Data added to the database successfully!")

Data added to the database successfully!


In [23]:
# inserting timdb movie ratings data in timdbratings table
df = pd.read_csv('C:/Users/karti/OneDrive/Desktop/IIA Project/data/timdb/ratings.csv')
# Calculate the average rating for each movie
average_ratings = df.groupby('movie_id')['rating'].mean().reset_index()

for i in range(len(average_ratings)):
    movieid = str(average_ratings['movie_id'][i])
    rating = average_ratings['rating'][i]
    rating = round(rating, 2)
    rating = float(rating)
    imdb_link = imdbid_dict[str(movieid)]
    try:
        dtuple = (movieid, imdb_link, rating)
        mycursor.execute(tirating_Query, dtuple)
        mydb.commit()
    except Exception as exc:
        print("Error is: ", exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [65]:
# inserting movielens movie data in mldbmovie table
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/movielens/movies.csv', 'r', encoding='utf8') as f:
    fopen = csv.reader(f, delimiter=',')
    for rdata in fopen:
        if rdata[0] == 'movieId':
            continue
        # Extracting data from the csv file
        movieid = rdata[0]
        title = rdata[1][:-7]
        re_d = rdata[1][-6:]
        release_date = re_d[1:-1]
        if movieid not in imdbid_dict:
            continue
            
        imdb_link = imdbid_dict[movieid]
        try:
            dtuple = (movieid, title, release_date, imdb_link)
            mycursor.execute(mlmovie_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ", exc)
print("Data added to the database successfully!")

Data added to the database successfully!


In [8]:
# inserting movielens movie ratings data in mldbbratings table
df = pd.read_csv('C:/Users/karti/OneDrive/Desktop/IIA Project/data/movielens/ratings.csv')
# Calculate the average rating for each movie
average_ratings = df.groupby('movieId')['rating'].mean().reset_index()

for i in range(len(average_ratings)):
    movieid = str(average_ratings['movieId'][i])
    rating = average_ratings['rating'][i]
    rating = round(rating, 2)
    rating = float(rating)
    if movieid not in imdbid_dict:
        continue
    imdb_link = imdbid_dict[str(movieid)]
    try:
        dtuple = (movieid, imdb_link, rating)
        mycursor.execute(mlrating_Query, dtuple)
        mydb.commit()
    except Exception as exc:
        print("Error is: ", exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [10]:
# inserting tmdb movies data in tmdbmovie table
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/tmdb/Movie_v11.csv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter=',')
    
    for rdata in fopen:
        if rdata[0] == 'id':
            continue
        imdbid_list = list(imdbid_dict.values())
        imdbid_list = [int(i) for i in imdbid_list]

        imdb_link = rdata[12]
        imdb_link = imdb_link[2:]
        if imdb_link == '':
            continue
        imdb_link = int(float(imdb_link))

        if imdb_link not in imdbid_list:
            continue
        title = rdata[1]
        budget = rdata[10]
        homepage_link = rdata[11]
        original_title = rdata[14]
        overview = rdata[15]
        popularity = rdata[16]
        poster_path = rdata[17]
        release_date = rdata[5]
        revenue = rdata[6]
        runtime = rdata[7]
        tagline = rdata[18]
        collection_belongs_to = rdata[20]
        is_adult = bool(rdata[8])

        # converting isAdult into binary representation for storing in database
        if is_adult is False:
            is_adult = 0
        else:
            is_adult = 1
            
        # converting runtime to int object for storing in database
        if runtime == '\\N':
            runtime = None

        try:
            dtuple = (title, original_title, release_date, runtime, is_adult, imdb_link, overview, popularity, poster_path, collection_belongs_to, revenue, budget, tagline, homepage_link)
            mycursor.execute(tmovie_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ",exc)
print("Data added to the database successfully!")

Data added to the database successfully!


In [16]:
# inserting tmdb person data in tmdbperson table

imdb_person_id = []
celeb_name = []

with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/tmdb/PERSON.tsv', 'r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')
    for rdata in fopen:
        if rdata[0] == 'ID':
            continue
        # Extracting data from the csv file
        if rdata[16] == 'IN' or rdata[17] == 'India' or rdata[18] == 'India':
            id = rdata[1]
            name = rdata[2]
            celeb_name.append(name)
            birthday = rdata[5]
            alias_name = rdata[6]
            deathday = rdata[7]
            gender = rdata[8]
            biography = rdata[9]
            profile_path = rdata[10]
            popularity = rdata[11]
            place_of_birth = rdata[12]
            imdb_link = rdata[14]
            imdb_person_id.append(imdb_link)
            homepage = rdata[15]
            home_country = "India"
            
            #converting birthday and deathday to none if they are not present 
            if birthday=='' or birthday=='\\N':
                birthday=None
            if deathday=='' or deathday=='\\N':
                deathday=None
                
            try:
                dtuple = (id, name, birthday, deathday, alias_name, gender, biography, profile_path, popularity, place_of_birth, homepage, home_country, imdb_link)
                mycursor.execute(tperson_Query, dtuple)
                mydb.commit()
            except Exception as exc:
                print("Error is: ",exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [ ]:
# # inserting rotten tomatoes movie data in rtmovie table

# with open('data/all_movie.csv', 'r',encoding='utf8') as f:
#     fopen = csv.reader(f, delimiter=',')

#     for rdata in fopen:
#         if rdata[0] == 'movie_id':
#             continue

#         # Extracting data from the csv file
#         cast1 = rdata[0]
#         cast2 = rdata[1]
#         description = rdata[6]
#         director = rdata[7]
#         genre = rdata[10]
#         rating = rdata[11]
#         title = rdata[15]
#         try:
#             dtuple = (title, cast1, cast2, description, director, genre, rating)
#             mycursor.execute(rtmovie_query, dtuple)
#             mydb.commit()
#         except Exception as exc:
#             print("Error is: ",exc)

# print("Data added to the database successfully!")

In [65]:
# inserting imdb movies data in imdbmovie table
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/imdb/title.basics.tsv', 'r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')
    for rdata in fopen:
        if rdata[0] == 'tconst':
            continue
        
        id = rdata[0]
        id = id[2:]
        if id == '':
            continue
        id = int(float(id))

        if id not in imdbid_list:
            continue
        
        primaryTitle = rdata[2]
        originalTitle = rdata[3]
        isAdult = rdata[4]
        realeaseDate = rdata[5]
        runtime = rdata[7]
        genres = rdata[8]
        
        # converting release date to datetime object for storing in database
        if realeaseDate != '\\N':
            realeaseDate = datetime.strptime(realeaseDate, '%Y')
        else:
            realeaseDate = datetime.strptime("2000", '%Y')
            
        # converting runtime to int object for storing in database
        if runtime != '\\N':
            runtime = int(runtime)
        else:
            runtime = None
            
        try:
            dtuple = (primaryTitle, originalTitle, realeaseDate, runtime, isAdult, id, genres)
            mycursor.execute(imovie_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ",exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [ ]:
# inserting imdb person data in imdbperson table
mid = 0
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/imdb/name.basics.tsv','r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')
    for rdata in fopen:
        if rdata[0] == 'nconst':
            continue
        # Extracting data from the csv file
        imdb_link = rdata[0]

        if imdb_link not in imdb_person_id:
            continue
        mid += 1
        name = rdata[1]
        birthday = rdata[2]
        deathday = rdata[3]
        primary_profession = rdata[4]
        known_for = rdata[5]
        
        #converting birthday and deathday to none if they are not present 
        if birthday=='' or birthday=='\\N':
            birthday=None
        if deathday=='' or deathday=='\\N':
            deathday=None
        try:
            dtuple = (mid, name, birthday, deathday, primary_profession, known_for, imdb_link)
            print(dtuple)
            mycursor.execute(iperson_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ",exc)
print("Data added to the database successfully!")

In [13]:
# inserting imdb movie ratings data in imdbratings table
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/imdb/title.ratings.tsv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter='\t')

    for rdata in fopen:
        if rdata[0] == 'tconst':
            continue
        
        id = rdata[0]
        id = id[2:]
        if id == '':
            continue
        id = int(float(id))

        if id not in imdbid_list:
            continue
        
        rating = rdata[1]
        votes = rdata[2]
        try:
            dtuple = (id, rating, votes)
            mycursor.execute(irating_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("Error is: ",exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [117]:
# creating a mapping between the ids of imdb, tmdb, timdb and movielens
index=0
with open('C:/Users/karti/OneDrive/Desktop/IIA Project/data/movielens/links.csv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter=',')

    for rdata in fopen:
        if rdata[0] == 'movieId':
            continue

        movieId = rdata[0]
        imdbId = rdata[1]
        tmdbId = rdata[2]
        id = int(float(imdbId))
        for key,value in imdbid_dict.items():
            temp = int(float(value))
            if temp == id:
                timdbId = key
                index+=1
                movieId=index
                try:
                    imdbId='tt'+imdbId
                    dtuple = (movieId, imdbId, tmdbId, timdbId)
                    mycursor.execute(dbmapping_Query, dtuple)
                    mydb.commit()
                except Exception as exc:
                    print("Error is: ",exc)

print("Data added to the database successfully!")

Data added to the database successfully!


In [46]:
# Making a call to Celebrity API to get information about celebrities using their name
def api_call(name):
    api_url = 'https://api.api-ninjas.com/v1/celebrity?name={}'.format(name)
    response = requests.get(api_url, headers={'X-Api-Key': 'I3HLYoEMA163se9Tu1LOZQ==2yaIidMHnjvzXREw'})
    if response.status_code == requests.codes.ok:
        txt = json.loads(response.text)
    else:
        print("Error:", response.status_code, response.text)
    return txt

In [54]:
for i in range(0,len(celeb_name)):

    txt = api_call(celeb_name[i])
    if txt==[] or len(txt[0]) < 5:
        continue

    name = txt[0]['name']
    if name in ['travis wear','savitri jindal','oksana baiul','sundar pichai','ajay piramal']:
        continue

    if txt[0]['birthday'] == "\\N" or txt[0]['birthday'] == '':
        txt[0]['birthday'] = None
    else:
        birthday = txt[0]['birthday']
        birthday = int(birthday[:4])
    
    occupation = txt[0]['occupation']
    list_as_string = ', '.join(occupation)
    # list_as_json = json.dumps(occupation)
    gender = txt[0]['gender']
    nationality = txt[0]['nationality']

    try:
        dtuple = (name, birthday, list_as_string, gender, nationality)
        mycursor.execute(capiperson_query, dtuple)
        mydb.commit()
    except Exception as exc:
        print("Error is: ",exc)

print("Data added to the database successfully!")
    


Data added to the database successfully!
